In [1]:
import polars as pl
import polars.selectors as cs
import numpy as np
import torch
import itertools
from concurrent.futures import ProcessPoolExecutor, as_completed
from loguru import logger
from typing import Tuple, Callable, Optional, Dict, List
from dataclasses import dataclass, field
from CONFIG import CONFIG
from FEATURE_ENGINEERING_SUBMISSION import FEATURE_ENGINEERING
from GBT_METRICS import CustomMetrics
from GBT_TRAINER import XGBoostTrainer, LightGBMTrainer
from GBT_OPTIMIZER import OptunaTuner
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit

import warnings

warnings.filterwarnings("ignore")


In [2]:
# data = pl.scan_csv(CONFIG.TRAIN_X_PATH, infer_schema_length=10_000).with_columns(pl.col(["risk_free_rate", "forward_returns", "market_forward_excess_returns"]).shift())

# feat = FEATURE_ENGINEERING(df=data)

# feat_engineered = feat.create_market_features().with_columns(pl.all().shrink_dtype())

# feat_engineered.write_parquet("data/wsl_feature_impt_train.parquet")

In [3]:
lgb_importance = pl.read_csv("data/LGBM_feature_importance.csv")
xgb_importance = pl.read_csv("data/XGB_feature_importance.csv")

In [4]:
(
    xgb_importance.with_columns((pl.col("shap_importance") / pl.col("shap_importance").sum()).cum_sum()).filter(
        (pl.col("shap_importance") <= 0.8) & (pl.col("rank") < 1_000)
    )
)["feature"].to_list()

['P9_V10_poly3_std_252_252',
 'E2_M3_exp_diff_std_252_90',
 'I2_M1_poly3_std_252_252',
 'M16_M8_log_ratio_std_252_10',
 'P2_std_252_10',
 'M16_M8_log_ratio_std_90_10',
 'E9_S10_sigmoid_diff_std_252_252',
 'M17_P10_poly3_std_252_90',
 'E19_std_90_10',
 'E3_P3_poly3_std_252_252',
 'P11_std_252_10',
 'V3_V7_exp_diff_std_90_90',
 'P10_sma_5_std_90_10',
 'P11_sma_5_std_252_10',
 'E3_M9_exp_diff_std_90_90',
 'M17_P10_poly2_std_252_252',
 'P13_P2_poly3_std_252_252',
 'E3_5_sortino_std_10_10',
 'M17_P2_exp_diff_std_90_90',
 'E2_E8_exp_diff_std_252_252',
 'P2_risk_free_rate_atan_ratio_std_252_252',
 'M17_P8_poly3_std_252_90',
 'M18_P11_poly3_std_10_10',
 'E9_S11_exp_diff_std_252_252',
 'E3_sma_10_std_10_10',
 'M18_P11_poly2_std_10_10',
 'P11_V8_log_mul_std_90_252',
 'E20_M1_diff_squared_std_252_90',
 'E3_V3_exp_diff_std_252_10',
 'P11_P7_poly3_std_252_10',
 'M8_V6_log_mul_std_10_10',
 'E19_V11_atan_ratio_std_252_10',
 'P10_sma_5_std_252_10',
 'E9_S9_sigmoid_diff_std_252_252',
 'E3_sma_10_std_90

In [5]:
(
    lgb_importance.join(xgb_importance, on="feature", suffix=("_xgb"))
    .with_columns(avg_impt=(pl.col("shap_importance") + pl.col("shap_importance_xgb")) / 2, avg_rank=(pl.col("rank") + pl.col("rank_xgb")) / 2)
    .sort(by="avg_impt", descending=True)
    .with_columns((pl.col("avg_impt") / pl.col("avg_impt").sum()).cum_sum())
    .filter((pl.col("avg_impt") <= 0.8) & (pl.col("avg_rank") < 1_000))
    .sort(by="avg_rank", descending=False)["feature"]
    .to_list()
)

['P2_std_252_10',
 'M17_P8_poly3_std_252_90',
 'P11_sma_5_std_252_10',
 'E2_E8_exp_diff_std_252_252',
 'E3_P11_harmonic_mean_std_252_90',
 'E9_S11_exp_diff_std_252_252',
 'E19_V11_atan_ratio_std_252_10',
 'E3_5_sortino_std_10_10',
 'M18_P11_poly3_std_10_10',
 'P11_P7_poly3_std_252_10',
 'M16_M8_log_ratio_std_252_10',
 'E19_252_sortino_std_10_10',
 'P10_sma_5_std_252_10',
 'E10_P2_exp_diff_std_10_10',
 'P11_V4_exp_diff_std_252_10',
 'E2_M18_poly3_std_10_10',
 'M16_M8_log_ratio_std_90_10',
 'P11_V2_exp_diff_std_90_90',
 'D3_E19_sigmoid_diff_std_252_90',
 'E3_sma_10_std_90_90',
 'M18_P11_poly2_std_10_10',
 'P2_sma_10_std_10_10',
 'P10_S9_atan_ratio_std_252_252',
 'P8_V13_diff_squared_std_90_10',
 'E3_sma_10_std_10_10',
 'M16_P11_log_mul_std_252_90',
 'M17_P11_log_mul_std_252_90',
 'E3_V3_sigmoid_diff_std_90_90',
 'E19_sma_5_std_90_90',
 'P11_V8_log_mul_std_90_252',
 'D9_E3_poly3_std_252_252',
 'M8_V9_exp_diff_std_252_90',
 'P2_P8_harmonic_mean_std_90_90',
 'P10_S6_poly3_std_90_252',
 'E19

In [6]:
data = pl.scan_csv(CONFIG.TRAIN_X_PATH, infer_schema_length=10_000).with_columns(
    pl.col(["risk_free_rate", "forward_returns", "market_forward_excess_returns"]).shift()
)

feat = FEATURE_ENGINEERING(df=data)

feat_engineered = feat.create_market_features()

train_x = feat_engineered.filter(pl.col(CONFIG.DATE_COL).is_between(CONFIG.START_DATE, CONFIG.END_DATE)).select(CONFIG.FEATURES)
train_y = pl.scan_csv(CONFIG.TRAIN_Y_PATH, infer_schema_length=10_000).filter(pl.col(CONFIG.DATE_COL).is_between(CONFIG.START_DATE, CONFIG.END_DATE))

risk_free_rate = (
    pl.scan_csv(CONFIG.TRAIN_X_PATH, infer_schema_length=10_000)
    .filter(pl.col(CONFIG.DATE_COL).is_between(CONFIG.START_DATE, CONFIG.END_DATE))
    .select(["risk_free_rate"])
    .collect()
    .to_numpy()
    .flatten()
)
forward_returns = (
    pl.scan_csv(CONFIG.TRAIN_X_PATH, infer_schema_length=10_000)
    .filter(pl.col(CONFIG.DATE_COL).is_between(CONFIG.START_DATE, CONFIG.END_DATE))
    .select(["forward_returns"])
    .collect()
    .to_numpy()
    .flatten()
)


test_x = feat_engineered.filter(pl.col(CONFIG.DATE_COL) > CONFIG.END_DATE).select(CONFIG.FEATURES)
test_y = pl.scan_csv(CONFIG.TRAIN_Y_PATH, infer_schema_length=10_000).filter(pl.col(CONFIG.DATE_COL) > CONFIG.END_DATE)
test_risk_free_rate = (
    pl.scan_csv(CONFIG.TRAIN_X_PATH, infer_schema_length=10_000)
    .filter(pl.col(CONFIG.DATE_COL) > CONFIG.END_DATE)
    .select(["risk_free_rate"])
    .collect()
    .to_numpy()
    .flatten()
)
test_forward_returns = (
    pl.scan_csv(CONFIG.TRAIN_X_PATH, infer_schema_length=10_000)
    .filter(pl.col(CONFIG.DATE_COL) > CONFIG.END_DATE)
    .select(["forward_returns"])
    .collect()
    .to_numpy()
    .flatten()
)


In [7]:
X = train_x.to_numpy()
y = train_y.drop(CONFIG.DATE_COL).collect().to_numpy().flatten()
test_x = test_x.to_numpy()
test_y = test_y.drop(CONFIG.DATE_COL).collect().to_numpy().flatten()
feature_names = train_x.columns

In [17]:
# Load study
import optuna
study = optuna.load_study(
    study_name="tune_v1",
    storage="sqlite:///tuning/v1/tune_v1.db"
)

# Get all trials as pandas DataFrame
df = study.trials_dataframe(attrs=("number", "value", "params", "state"))
df.sort_values(by='value')

,number,value,params_alpha,params_batch_size,params_colsample_bylevel,params_colsample_bytree,params_eta,params_gamma,params_lambda,params_max_depth,params_min_child_weight,params_num_boost_round,params_subsample,state
3,3,0.104584,0.132446,110,0.873693,0.639069,0.088897,1.485531,0.275520,10,7,1000,0.721846,COMPLETE
24,24,0.161374,0.561508,350,0.700173,0.689600,0.052860,0.022498,0.348474,5,5,500,0.600138,COMPLETE
8,8,0.273897,0.311742,270,0.824511,0.904314,0.077128,1.568198,0.590156,7,2,100,0.885298,COMPLETE
25,25,0.277467,0.888233,310,0.849457,0.760099,0.069646,0.346532,0.538198,8,4,400,0.744507,COMPLETE
18,18,0.321918,0.484184,450,0.642769,0.785716,0.099373,0.794130,0.329119,8,6,200,0.766954,COMPLETE
26,26,0.333330,0.391682,370,0.778465,0.636865,0.044466,0.779016,0.731644,7,3,800,0.641491,COMPLETE
21,21,0.344860,0.490957,280,0.609150,0.732538,0.040752,0.081015,0.445302,8,4,400,0.601829,COMPLETE
16,16,0.350492,0.270598,240,0.712078,0.700462,0.031920,0.972130,0.809063,6,8,400,0.658459,COMPLETE
9,9,0.355401,0.257266,190,0.963027,0.803428,0.012820,2.266653,0.177538,4,6,100,0.725742,COMPLETE
2,2,0.361735,0.111289,160,0.805694,0.679870,0.019595,1.822635,0.391214,6,5,100,0.914070,COMPLETE


In [18]:
study.best_params

{'eta': 0.09330606024425668,
 'max_depth': 9,
 'min_child_weight': 3,
 'subsample': 0.6727299868828402,
 'colsample_bytree': 0.6733618039413735,
 'colsample_bylevel': 0.7216968971838151,
 'lambda': 0.3347776308515933,
 'alpha': 0.2703616066662001,
 'gamma': 0.8736874205941257,
 'batch_size': 350,
 'num_boost_round': 200}

In [15]:
optuna.visualization.plot_param_importances(study)

In [8]:
tune = OptunaTuner(
    X=X,
    y=y,
    final_x_test=test_x,
    risk_free_rate=risk_free_rate,
    forward_returns=forward_returns,
    final_risk_free_rate=test_risk_free_rate,
    final_forward_returns=test_forward_returns,
    feature_names=feature_names,
)
study = tune.optimize(study_name="tune_v1", folder="tuning/v1")

[I 2025-11-16 18:27:40,239] A new study created in RDB with name: tune_v1


  0%|          | 0/30 [00:00<?, ?it/s]

[0]	val-adj_sharpe:0.38218
[10]	val-adj_sharpe:0.35985
[20]	val-adj_sharpe:0.33372
[30]	val-adj_sharpe:0.33673
[40]	val-adj_sharpe:0.32979
[50]	val-adj_sharpe:0.31382
[60]	val-adj_sharpe:0.30251
[70]	val-adj_sharpe:0.30400
[80]	val-adj_sharpe:0.29902
[90]	val-adj_sharpe:0.28149
[99]	val-adj_sharpe:0.27935


2025-11-16 18:27:41.115 | INFO     | GBT_TRAINER:train_cv:127 - INCREMENTAL LEARNING STARTED


[0]	val-adj_sharpe:-0.04465
[10]	val-adj_sharpe:-0.00449
[20]	val-adj_sharpe:0.09847
[30]	val-adj_sharpe:0.03693
[40]	val-adj_sharpe:-0.00556
[50]	val-adj_sharpe:-0.02127
[60]	val-adj_sharpe:-0.01991
[70]	val-adj_sharpe:-0.02755
[80]	val-adj_sharpe:-0.03184
[90]	val-adj_sharpe:-0.05205
[99]	val-adj_sharpe:-0.05281


...

[0]	val-adj_sharpe:0.45455
[10]	val-adj_sharpe:0.44769
[20]	val-adj_sharpe:0.45977
[30]	val-adj_sharpe:0.44306
[40]	val-adj_sharpe:0.43032
[50]	val-adj_sharpe:0.42822
[60]	val-adj_sharpe:0.42387
[70]	val-adj_sharpe:0.44303
[80]	val-adj_sharpe:0.43199
[90]	val-adj_sharpe:0.42695
[100]	val-adj_sharpe:0.42789
[110]	val-adj_sharpe:0.41667
[116]	val-adj_sharpe:0.42286


2025-11-16 19:58:35.359 | INFO     | GBT_TRAINER:train_cv:127 - INCREMENTAL LEARNING STARTED


[0]	val-adj_sharpe:0.49366
[10]	val-adj_sharpe:0.51215
[20]	val-adj_sharpe:0.53261
[30]	val-adj_sharpe:0.54207
[40]	val-adj_sharpe:0.55254
[50]	val-adj_sharpe:0.54126
[60]	val-adj_sharpe:0.52741
[70]	val-adj_sharpe:0.54581
[80]	val-adj_sharpe:0.54304
[90]	val-adj_sharpe:0.54126
[100]	val-adj_sharpe:0.54674
[110]	val-adj_sharpe:0.55256
[120]	val-adj_sharpe:0.56534
[130]	val-adj_sharpe:0.56908
[140]	val-adj_sharpe:0.59256
[150]	val-adj_sharpe:0.59923
[160]	val-adj_sharpe:0.60338
[170]	val-adj_sharpe:0.59268
[180]	val-adj_sharpe:0.60735
[190]	val-adj_sharpe:0.60012
[200]	val-adj_sharpe:0.58902
[210]	val-adj_sharpe:0.59020
[220]	val-adj_sharpe:0.59109
[230]	val-adj_sharpe:0.60135
[240]	val-adj_sharpe:0.59052
[250]	val-adj_sharpe:0.59353
[260]	val-adj_sharpe:0.59743
[270]	val-adj_sharpe:0.59350
[278]	val-adj_sharpe:0.59649


2025-11-16 19:59:15.348 | INFO     | GBT_TRAINER:train_cv:127 - INCREMENTAL LEARNING STARTED
2025-11-16 19:59:29.529 | INFO     | GBT_TRAINER:train_cv:180 - FINAL_TEST STARTED
2025-11-16 19:59:32.598 | INFO     | GBT_TRAINER:train_cv:225 - Avg: 0.2436446034853935 | final test score: ('adj_sharpe', 0.4045552879474013)
2025-11-16 19:59:32.598 | INFO     | GBT_TRAINER:get_results_summary:48 - Mean Test Sharpe: 0.2436 (+/- 0.0919)
2025-11-16 19:59:32.648 | INFO     | GBT_OPTIMIZER:optimize:120 - Best trial value: 0.6387
2025-11-16 19:59:32.648 | INFO     | GBT_OPTIMIZER:optimize:121 - Best trial params:
2025-11-16 19:59:32.648 | INFO     | GBT_OPTIMIZER:optimize:123 -   eta: 0.09330606024425668
2025-11-16 19:59:32.649 | INFO     | GBT_OPTIMIZER:optimize:123 -   max_depth: 9
2025-11-16 19:59:32.650 | INFO     | GBT_OPTIMIZER:optimize:123 -   min_child_weight: 3
2025-11-16 19:59:32.650 | INFO     | GBT_OPTIMIZER:optimize:123 -   subsample: 0.6727299868828402
2025-11-16 19:59:32.650 | INFO   

[I 2025-11-16 19:59:32,638] Trial 29 finished with value: 0.4045552879474013 and parameters: {'eta': 0.028164727631512586, 'max_depth': 10, 'min_child_weight': 4, 'subsample': 0.6298940305316926, 'colsample_bytree': 0.7179413424458323, 'colsample_bylevel': 0.6386381995119802, 'lambda': 0.7177163028650332, 'alpha': 0.5468529343775985, 'gamma': 0.19523303130633782, 'batch_size': 330, 'num_boost_round': 400}. Best is trial 1 with value: 0.6386943273525385.


In [7]:
xgb_trainer = XGBoostTrainer(params=CONFIG.temp)
xgb_res = xgb_trainer.train_cv(
    X=X,
    y=y,
    final_x_test=test_x,
    risk_free_rate=risk_free_rate,
    forward_returns=forward_returns,
    final_risk_free_rate=test_risk_free_rate,
    final_forward_returns=test_forward_returns,
    batch_size=CONFIG.temp.get("batch_size"),
    NUM_BOOST_ROUND=CONFIG.temp.get("num_boost_round"),
    folder="submission_models/test1",
)

[0]	val-rmse:0.30267	val-adj_sharpe:0.35289
[10]	val-rmse:0.34735	val-adj_sharpe:0.45715
[20]	val-rmse:0.34979	val-adj_sharpe:0.46611
[30]	val-rmse:0.35384	val-adj_sharpe:0.42841
[40]	val-rmse:0.35348	val-adj_sharpe:0.42782
[50]	val-rmse:0.35533	val-adj_sharpe:0.38653
[60]	val-rmse:0.35482	val-adj_sharpe:0.40459
[70]	val-rmse:0.35510	val-adj_sharpe:0.40503
[80]	val-rmse:0.35508	val-adj_sharpe:0.40500
[90]	val-rmse:0.35718	val-adj_sharpe:0.42393
[100]	val-rmse:0.35579	val-adj_sharpe:0.42163
[109]	val-rmse:0.36059	val-adj_sharpe:0.38987


2025-11-16 16:34:42.316 | INFO     | GBT_TRAINER:train_cv:127 - INCREMENTAL LEARNING STARTED


[0]	val-rmse:0.47723	val-adj_sharpe:-0.02772
[10]	val-rmse:0.49918	val-adj_sharpe:0.41584
[20]	val-rmse:0.50734	val-adj_sharpe:0.41934
[30]	val-rmse:0.50786	val-adj_sharpe:0.40788
[40]	val-rmse:0.50748	val-adj_sharpe:0.40984
[50]	val-rmse:0.51589	val-adj_sharpe:0.36001
[60]	val-rmse:0.51544	val-adj_sharpe:0.36185
[70]	val-rmse:0.52164	val-adj_sharpe:0.30001
[80]	val-rmse:0.52117	val-adj_sharpe:0.30152
[90]	val-rmse:0.52117	val-adj_sharpe:0.30154
[100]	val-rmse:0.52122	val-adj_sharpe:0.30136
[109]	val-rmse:0.52109	val-adj_sharpe:0.30266


2025-11-16 16:35:48.720 | INFO     | GBT_TRAINER:train_cv:127 - INCREMENTAL LEARNING STARTED


[0]	val-rmse:0.48305	val-adj_sharpe:-0.15853
[10]	val-rmse:0.48030	val-adj_sharpe:0.10315
[20]	val-rmse:0.48028	val-adj_sharpe:0.08993
[30]	val-rmse:0.48253	val-adj_sharpe:0.14362
[40]	val-rmse:0.48236	val-adj_sharpe:0.14907
[50]	val-rmse:0.48235	val-adj_sharpe:0.14802
[60]	val-rmse:0.48238	val-adj_sharpe:0.15190
[70]	val-rmse:0.48238	val-adj_sharpe:0.15159
[80]	val-rmse:0.48240	val-adj_sharpe:0.15353
[90]	val-rmse:0.48241	val-adj_sharpe:0.15465
[100]	val-rmse:0.48240	val-adj_sharpe:0.15356
[109]	val-rmse:0.48239	val-adj_sharpe:0.15287


2025-11-16 16:36:40.038 | INFO     | GBT_TRAINER:train_cv:127 - INCREMENTAL LEARNING STARTED


[0]	val-rmse:0.55926	val-adj_sharpe:0.42984
[10]	val-rmse:0.55498	val-adj_sharpe:0.44745
[20]	val-rmse:0.55699	val-adj_sharpe:0.42984
[30]	val-rmse:0.55859	val-adj_sharpe:0.40946
[40]	val-rmse:0.55900	val-adj_sharpe:0.40281
[50]	val-rmse:0.55900	val-adj_sharpe:0.40341
[60]	val-rmse:0.55890	val-adj_sharpe:0.39536
[70]	val-rmse:0.55870	val-adj_sharpe:0.38427
[80]	val-rmse:0.55861	val-adj_sharpe:0.37762
[90]	val-rmse:0.55861	val-adj_sharpe:0.37780
[100]	val-rmse:0.55861	val-adj_sharpe:0.37741
[102]	val-rmse:0.55861	val-adj_sharpe:0.37753


2025-11-16 16:37:18.609 | INFO     | GBT_TRAINER:train_cv:127 - INCREMENTAL LEARNING STARTED


[0]	val-rmse:0.56886	val-adj_sharpe:0.43057
[10]	val-rmse:0.57483	val-adj_sharpe:0.49554
[20]	val-rmse:0.57529	val-adj_sharpe:0.46779
[30]	val-rmse:0.57521	val-adj_sharpe:0.46401
[40]	val-rmse:0.57513	val-adj_sharpe:0.46449
[50]	val-rmse:0.57518	val-adj_sharpe:0.46106
[60]	val-rmse:0.57526	val-adj_sharpe:0.46660
[70]	val-rmse:0.57520	val-adj_sharpe:0.46693
[80]	val-rmse:0.57520	val-adj_sharpe:0.46692
[90]	val-rmse:0.57526	val-adj_sharpe:0.46657
[100]	val-rmse:0.57525	val-adj_sharpe:0.46665
[107]	val-rmse:0.57513	val-adj_sharpe:0.46736


2025-11-16 16:37:44.767 | INFO     | GBT_TRAINER:train_cv:127 - INCREMENTAL LEARNING STARTED
2025-11-16 16:37:58.057 | INFO     | GBT_TRAINER:train_cv:186 - FINAL_TEST STARTED
2025-11-16 16:38:00.673 | INFO     | GBT_TRAINER:train_cv:234 - Avg: 0.390930712916657 | final test score: ('adj_sharpe', 1.7381347593427718)
2025-11-16 16:38:00.677 | INFO     | GBT_TRAINER:get_results_summary:48 - Mean Test Sharpe: 0.3909 (+/- 0.0929)


In [27]:
lgb_trainer = LightGBMTrainer()
lgb_res = lgb_trainer.train_cv(
    X=X,
    y=y,
    final_x_test=test_x,
    risk_free_rate=risk_free_rate,
    forward_returns=forward_returns,
    final_risk_free_rate=test_risk_free_rate,
    final_forward_returns=test_forward_returns,
    batch_size=CONFIG.temp.get("batch_size"),
    NUM_BOOST_ROUND=CONFIG.temp.get("num_boost_round"),
    folder="submission_models/test",
)

2025-11-16 00:59:53.415 | INFO     | GBT_TRAINER:train_cv:319 - INCREMENTAL LEARNING STARTED


Training until validation scores don't improve for 100 rounds
[10]	validation's adj_sharpe: 0.30999
[20]	validation's adj_sharpe: 0.297199
[30]	validation's adj_sharpe: 0.278889
[40]	validation's adj_sharpe: 0.206482
[50]	validation's adj_sharpe: 0.19537
[60]	validation's adj_sharpe: 0.178418
[70]	validation's adj_sharpe: 0.168368
[80]	validation's adj_sharpe: 0.16857
[90]	validation's adj_sharpe: 0.145489
[100]	validation's adj_sharpe: 0.125904
Early stopping, best iteration is:
[1]	validation's adj_sharpe: 0.36543


2025-11-16 01:00:55.379 | INFO     | GBT_TRAINER:train_cv:355 - FINAL_TEST STARTED
2025-11-16 01:00:57.944 | INFO     | GBT_TRAINER:train_cv:319 - INCREMENTAL LEARNING STARTED


Training until validation scores don't improve for 100 rounds
[10]	validation's adj_sharpe: 0.0689714
[20]	validation's adj_sharpe: 0.235914
[30]	validation's adj_sharpe: 0.346363
[40]	validation's adj_sharpe: 0.355692
[50]	validation's adj_sharpe: 0.3706
[60]	validation's adj_sharpe: 0.371446
[70]	validation's adj_sharpe: 0.399971
[80]	validation's adj_sharpe: 0.319675
[90]	validation's adj_sharpe: 0.361499
[100]	validation's adj_sharpe: 0.38649
[110]	validation's adj_sharpe: 0.38166
Did not meet early stopping. Best iteration is:
[104]	validation's adj_sharpe: 0.421726


2025-11-16 01:01:49.590 | INFO     | GBT_TRAINER:train_cv:355 - FINAL_TEST STARTED
2025-11-16 01:01:52.397 | INFO     | GBT_TRAINER:train_cv:319 - INCREMENTAL LEARNING STARTED


...

2025-11-16 01:02:31.157 | INFO     | GBT_TRAINER:train_cv:355 - FINAL_TEST STARTED
2025-11-16 01:02:33.862 | INFO     | GBT_TRAINER:train_cv:319 - INCREMENTAL LEARNING STARTED


Training until validation scores don't improve for 100 rounds
[10]	validation's adj_sharpe: 0.372507
[20]	validation's adj_sharpe: 0.300263
[30]	validation's adj_sharpe: 0.308181
[40]	validation's adj_sharpe: 0.274678
[50]	validation's adj_sharpe: 0.267665
[60]	validation's adj_sharpe: 0.252252
[70]	validation's adj_sharpe: 0.245632
[80]	validation's adj_sharpe: 0.239534
[90]	validation's adj_sharpe: 0.260419
[100]	validation's adj_sharpe: 0.25948
Early stopping, best iteration is:
[2]	validation's adj_sharpe: 0.440589


2025-11-16 01:02:59.154 | INFO     | GBT_TRAINER:train_cv:355 - FINAL_TEST STARTED
2025-11-16 01:03:01.869 | INFO     | GBT_TRAINER:train_cv:319 - INCREMENTAL LEARNING STARTED


Training until validation scores don't improve for 100 rounds
[10]	validation's adj_sharpe: 0.596762
[20]	validation's adj_sharpe: 0.688382
[30]	validation's adj_sharpe: 0.684981
[40]	validation's adj_sharpe: 0.674441
[50]	validation's adj_sharpe: 0.619852
[60]	validation's adj_sharpe: 0.643368
[70]	validation's adj_sharpe: 0.627966
[80]	validation's adj_sharpe: 0.580521
[90]	validation's adj_sharpe: 0.607627
[100]	validation's adj_sharpe: 0.628525
[110]	validation's adj_sharpe: 0.639279
Did not meet early stopping. Best iteration is:
[34]	validation's adj_sharpe: 0.714979


2025-11-16 01:03:15.357 | INFO     | GBT_TRAINER:train_cv:355 - FINAL_TEST STARTED
2025-11-16 01:03:17.807 | INFO     | GBT_TRAINER:train_cv:421 - Avg: 0.3716426261875769 | final test avg: 0.45416761958263335
2025-11-16 01:03:17.807 | INFO     | GBT_TRAINER:get_results_summary:48 - Mean Test Sharpe: 0.3716 (+/- 0.0646)
